### Pre-process annotated airline tweet data provided by Crowdflower
A Super Handy CrowdFlower Glossary of Terms can be found [here](https://success.crowdflower.com/hc/en-us/articles/202703305-Glossary-of-Terms)!

In [1]:
import numpy as np
import pandas as pd

#### Read-In Jobs-Level Data (from CrowdFlower's *Data for Everyone* [library](https://www.crowdflower.com/data-for-everyone/))

In [2]:
cf = pd.read_csv("http://cdn2.hubspot.net/hub/346378/file-2612489700-csv/DFE_CSVs/Airline-Full-Non-Ag-DFE-Sentiment.csv")
print cf.columns
cf.head(2)

Index([u'_unit_id', u'_created_at', u'_golden', u'_id', u'_missed',
       u'_started_at', u'_tainted', u'_channel', u'_trust', u'_worker_id',
       u'_country', u'_region', u'_city', u'_ip', u'airline_sentiment',
       u'negativereason', u'airline', u'airline_sentiment_gold', u'name',
       u'negativereason_gold', u'retweet_count', u'text', u'tweet_coord',
       u'tweet_created', u'tweet_id', u'tweet_location', u'user_timezone'],
      dtype='object')


,_unit_id,_created_at,_golden,_id,_missed,_started_at,_tainted,_channel,_trust,_worker_id,...,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_id,tweet_location,user_timezone
0,681448150,2/25/2015 04:52:40,False,1575073003,NaN,2/25/2015 04:49:12,False,elite,0.8108,31110645,...,NaN,cairdin,NaN,0,@VirginAmerica What @dhepburn said.,NaN,2015-02-24 11:35:52 -0800,570306133677760513,NaN,Eastern Time (US & Canada)
1,681448150,2/25/2015 05:22:10,False,1575093916,NaN,2/25/2015 05:19:59,False,prodege,0.8919,1908948,...,NaN,cairdin,NaN,0,@VirginAmerica What @dhepburn said.,NaN,2015-02-24 11:35:52 -0800,570306133677760513,NaN,Eastern Time (US & Canada)


#### Split Test (i.e. Golden Tweets) out from the non-test tweets.
* (We already know the "correct" answers for the Test tweets, so we can process those seperately.)
* Also, per CF's recommendation, we only keep judegements from contibutors with a trust score of at least 70%

In [7]:
gold =   cf[cf._golden==True].copy()
nogold = cf[(cf._golden==False) & (cf._trust >= 0.70)].copy()

print gold.shape, nogold.shape

(11997, 27) (43786, 27)


#### Create 0/1 sentiment indicator for non-test tweets

In [8]:
nogold.airline_sentiment.value_counts(dropna=False)

negative    26919
neutral      9742
positive     7125
Name: airline_sentiment, dtype: int64

In [10]:
nogold["complaint0"] = 0
nogold["complaint1"] = 0
nogold["judgements"] = 1

nogold["complaint0trust"] = np.nan
nogold["complaint1trust"] = np.nan

nogold.ix[nogold["airline_sentiment"].isin(["positive","neutral"]), "complaint0"] = 1
nogold.ix[nogold["airline_sentiment"]=="negative", "complaint1"] = 1

nogold.ix[nogold["airline_sentiment"].isin(["positive","neutral"]), "complaint0trust"] = nogold._trust
nogold.ix[nogold["airline_sentiment"]=="negative", "complaint1trust"] = nogold._trust

In [12]:
nogold[["complaint0","complaint1","complaint0trust","complaint1trust","_trust"]].head(5)

,complaint0,complaint1,complaint0trust,complaint1trust,_trust
0,1,0,0.8108,NaN,0.8108
1,1,0,0.8919,NaN,0.8919
2,1,0,0.7222,NaN,0.7222
3,1,0,0.8966,NaN,0.8966
4,1,0,0.8108,NaN,0.8108


In [21]:
c_flags = nogold.groupby(by=["tweet_id", "text"], as_index=False)[["complaint0","complaint1"]].max()
c_total = nogold.groupby(by=["tweet_id", "text"], as_index=False)["judgements"].sum()
c_trust = nogold.groupby(by=["tweet_id", "text"], as_index=False)[["complaint0trust","complaint1trust"]].mean()

complaints = pd.merge(c_flags, c_total, on=["tweet_id", "text"]) ##try PD concat instead

TypeError: Could not compare ['left'] with block values

In [19]:
c_trust.head()

,tweet_id,text,complaint0trust,complaint1trust
0,567588278875213824,@JetBlue's new CEO seeks the right balance to ...,0.9009,NaN
1,567590027375702016,@JetBlue is REALLY getting on my nerves !! 😡�...,NaN,0.916467
2,567591480085463040,@united yes. We waited in line for almost an h...,NaN,0.859233
3,567592368451248130,@united the we got into the gate at IAH on tim...,NaN,0.840500
4,567594449874587648,@SouthwestAir its cool that my bags take a bit...,NaN,0.835267


In [ ]:
complaints[complaints.total==2].head(20)

##### Classify Labels

In [ ]:
def define_complaints(sentiment, complaint):
    if sentiment in ["positive", "neutral"]:
        return "No Complaint"
    elif complaint in ["Cancelled Flight", "Late Flight"]:
        return "Delay or Cancellation"
    elif complaint in ["Lost Luggage", "Damaged Luggage"]:
        return "Lost or Damaged Luggage"
    elif complaint in ["Customer Service Issue", "Flight Attendant Complaints", "Flight Booking Problems", "longlines"]:
        return "Customer Service"
    elif compaint in ["Bad Flight", "Can't Tell"]:
        return "Unknown"
    
cf["complaint"] = cf[["airline_sentiment", "negativereason"]].apply(define_complaints)

#### Read In Crowdflower Data

#### CrowdFlower Data Preprocessing